In [101]:
import sys
# sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
# sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [102]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tk = th.Thicket.from_caliperreader(glob("califiles/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 279/279 [00:02<00:00, 108.23it/s]


View Calltree

In [88]:
print(tk.tree(metric_column="Max time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

0.579 main
├─ 0.017 MPI_Comm_dup
├─ 0.000 MPI_Finalize
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
├─ 0.000 MPI_Initialized
└─ 0.202 main
   ├─ 0.049 comm
   │  ├─ 0.044 comm_large
   │  │  ├─ 0.017 MPI_Alltoall
   │  │  └─ 0.029 MPI_Scatterv
   │  └─ 0.008 comm_small
   │     ├─ 0.008 MPI_Bcast
   │     └─ 0.005 MPI_Gather
   ├─ 0.132 comp
   │  ├─ 0.131 comp_large
   │  └─ 0.000 comp_small
   ├─ 0.037 correctness_check
   │  ├─ 0.000 MPI_Gather
   │  └─ 0.037 MPI_Gatherv
   └─ 0.019 data_init_runtime

Legend (Metric: Max time/rank Min: 0.00 Max: 0.58 indices: {'profile': 4198114})
█ 0.52 - 0.58
█ 0.41 - 0.52
█ 0.29 - 0.41
█ 0.17 - 0.29
█ 0.06 - 0.17
█ 0.00 - 0.06

name User code    ◀  Only in left graph    ▶  Only in right graph



Group Performance data by `matrix_size` in the Thicket metadata table.

In [103]:
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "num_procs", "input_size", "input_type"]).sort_index()

tk.dataframe.head(20)

profile  \
node                                 num_procs input_size input_type                             
{'name': 'main', 'type': 'function'} 2         65536      Random                    3669930900   
                                                          Reverse Sorted            2415705447   
                                                          Sorted                     300411230   
                                                          Sorted with 1% perturbed   972226623   
                                               262144     Random                    3402972926   
                                                          Reverse Sorted            1198010761   
                                                          Sorted                    2108853330   
                                                          Sorted with 1% perturbed  3003965217   
                                               1048576    Random                     440470322   
                                                          Reverse Sorted            3816414986   
                                                          Sorted                    1536058806   
                                                          Sorted with 1% perturbed   261843834   
                                               4194304    Random                     603037390   
                                                          Reverse Sorted            2852939213   
                                                          Sorted                    3523986067   
                                                          Sorted with 1% perturbed   884934137   
                                               16777216   Random                     477269477   
                                                          Reverse Sorted            3036698840   
                                                          Sorted                     875396164   
                                                          Sorted with 1% perturbed   655752124   

                                                                                    nid  \
node                                 num_procs input_size input_type                      
{'name': 'main', 'type': 'function'} 2         65536      Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               262144     Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               1048576    Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               4194304    Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               16777216   Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                   

In [98]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
input_sizes = [2**i for i in range(16, 30, 2)]
input_types = ["Random", "Sorted", "Reverse Sorted", "Sorted with 1% perturbed"]

In [91]:
# Change font size for all plots
plt.rcParams.update({"font.size": 20})

In [92]:
# check what's missing out of all combinations of processes, input sizes, and input types
allRealCombinations = []
for row in tk.dataframe.index:
    allRealCombinations.append((row[1], row[2], row[3]))

missing = []
for p in processes:
    for i in input_sizes:
        for t in input_types:
            if (p, i, t) not in allRealCombinations:
                missing.append((p, i, t))
missing.sort()
print(missing)

# get all unique number of processes and input sizes from missing
missingProcesses = set([m[0] for m in missing])
missingInputSizes = set([m[1] for m in missing])

# combine missing processes and input sizes
missingCombinations = []
for p in missingProcesses:
    for i in missingInputSizes:
        missingCombinations.append((p, i))

# missingCombinations.sort()
# print(missingCombinations)
# print(len(missing))



[]


In [122]:
def create_graphs(df):
    input_sizes = df.index.get_level_values('input_size').unique()
    num_procs = df.index.get_level_values('num_procs').unique()
    nodeVals = ['comm', 'comp']
    print(input_sizes)
    for node in nodeVals:
        for size in input_sizes:
            fig, axs = plt.subplots(2, 3, figsize=(20, 15))
            fig.suptitle(f'Performance Metrics {size} - {node}')
            size_data = df[df.index.get_level_values('input_size') == size]
            axis = [(0,0), (0, 1), (0,2), (1, 0), (1, 1)]
            metrics = ['Min time/rank', 'Max time/rank', "Avg time/rank", "Total time", "Variance time/rank"]
            handles, labels = [], []
            for i, metric in enumerate(metrics):
                ax = axs[axis[i]]
                ax.set_title(metric)
                for input_type in input_types:
                    input_data = size_data[size_data.index.get_level_values('input_type') == input_type]
                    tempList = [node.frame.get("name") for node in input_data.index.get_level_values("node").values]
                    nodeLoc = input_data.index.get_level_values("node").values[tempList.index(node)]
                    data = input_data.loc[nodeLoc]
                    
                    if metric == "Total time":
                        # divide each point by the number of processes
                        data[metric] = data[metric] / data.index.get_level_values('num_procs')
                    ax.plot(num_procs, data[metric], label=input_type)
                ax.set_xlabel('Number of Processes')
                ax.set_ylabel('Time (s)')
                ax.set_xscale('log', base=2)
                handles, labels = ax.get_legend_handles_labels()
            
            axs[1, 2].axis('off')
            ax = axs[1, 2]
            
            ax.legend(handles, labels)
            plt.savefig(f'performance_metrics_{size}_{node}.png')
            plt.close()
            

    # node = 'main'
    # for input_type in input_types:
    #     fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    #     fig.suptitle(f'Performance Metrics {size}', fontsize=16)
    #     input_data = df[df.index.get_level_values('input_type') == input_type]
    #     axis = [(0,0), (0, 1), (1, 0), (1, 1)]
    #     metrics = ['Min time/rank (exc)', 'Max time/rank (exc)', "Avg time/rank (exc)", "Total time"]
    #     for i, metric in enumerate(metrics):
    #         ax = axs[axis[i]]
    #         ax.set_title(metric)
    #         for size in input_sizes:
    #             size_data = input_data[input_data.index.get_level_values('input_size') == size]
    #             tempList = [node.frame.get("name") for node in size_data.index.get_level_values("node").values]
    #             nodeLoc = size_data.index.get_level_values("node").values[tempList.index(node)]
    #             data = size_data.loc[nodeLoc]
    #             ax.plot(num_procs, data[metric], label=size)
    #         ax.set_xlabel('Number of Processes')
    #         ax.set_ylabel('Time (s)')
    #         ax.legend()
    #     plt.savefig(f'performance_metrics_{input_type}_{node}.png')
    #     plt.close()
# Create graphs
create_graphs(tk.dataframe)

Index([65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456], dtype='int64', name='input_size')


In [ ]:
# fig, axs = plt.subplots(2, 2, figsize=(20, 15))
#         fig.suptitle(f'Performance Metrics {size}x{size}', fontsize=16)
#         size_data = df[df.index.get_level_values('matrix_size') == size]

#         ax = axs[0, 0]
#         operations = ['whole_computation', 'master_initialization', 'master_send_recieve']
#         for op in operations:
#             tempList = [node.frame.get("name") for node in size_data.index.get_level_values("node").values]
#             nodeLoc = size_data.index.get_level_values("node").values[tempList.index(op)]
#             data = size_data.loc[nodeLoc]
#             ax.plot(data.index.get_level_values('num_procs'), data['Total time'], 
#                     marker='o', label=f"{op} (size {size})")
#         ax.set_xlabel('Number of Processes')
#         ax.set_ylabel('Total Time (s)')
#         ax.set_title('Total Time for Different Operations')
#         # ax.set_xscale('log', base=2)
#         # ax.set_yscale('log')
#         ax.legend()
        
#         worker_functions = ['worker_calculation', 'worker_recieve', 'worker_send']
#         metrics = ['Min time/rank (exc)', 'Max time/rank (exc)', 'Avg time/rank (exc)']
#         
        
#         for i, metric in enumerate(metrics):
#             ax = axs[axis[i]]
#             for func in worker_functions:
#                 tempList = [node.frame.get("name") for node in size_data.index.get_level_values("node").values]
#                 nodeLoc = size_data.index.get_level_values("node").values[tempList.index(func)]
#                 data = size_data.loc[nodeLoc]
                
                    
#                 ax.plot(data.index.get_level_values('num_procs'), data[metric], 
#                         marker='o', label=f"{func} (size {size})")
#             ax.set_xlabel('Number of Processes')
#             ax.set_ylabel(f'{metric} (s)')
#             ax.set_title(f'Worker Functions - {metric}')
#             # ax.set_xscale('log', base=2)
#             # ax.set_yscale('log')
#             ax.legend()
        
#         plt.tight_layout()
#         plt.savefig(f'performance_graphs{size}.png')
#         plt.close()